# 데이터 수집

- [프레임셋 전환](http://www.idom.kr/bbs/board.php?bo_table=E_table03&wr_id=125)


- [크롤링 태그 찾기](https://desarraigado.tistory.com/14)


- [팝업창 제어](https://m.blog.naver.com/tkdtjdtkddms/221976262160)

## (1) Web crawling test under the tag of framset

In [1]:
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
url = 'https://www.rm1.co.kr/'

In [3]:
# Chrome Webdriver 실행:
driver = webdriver.Chrome(r"./chromedriver.exe")

C:\Users\achil\AppData\Local\Temp\ipykernel_19360\3780290278.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"./chromedriver.exe")


#### Case 1: 

In [4]:
driver.get(url)
html = driver.page_source
html

'<html><head>\n\t<meta name="google-site-verification" content="uvLrphPZotcDpspdcgkCoo8NypsIGl309A67wMoK0qY">\n</head>\n<frameset rows="100%,*">\n\t<frame src="/p00/A0001M002.nice">\n</frameset>\n</html>'

In [5]:
BeautifulSoup(html, 'html.parser')

<html><head>
<meta content="uvLrphPZotcDpspdcgkCoo8NypsIGl309A67wMoK0qY" name="google-site-verification"/>
</head>
<frameset rows="100%,*">
<frame src="/p00/A0001M002.nice"/>
</frameset>
</html>

#### Case 2: 

 - frameset 태그

In [6]:
url_frame = url + '/p00/A0001M002.nice'

In [7]:
driver.get(url_frame)
html = driver.page_source

In [8]:
# BeautifulSoup(html, 'html.parser')
# BeautifulSoup(html, 'lxml')

#### Case 3: 개선사례

In [9]:
driver.get(url)
html = driver.page_source; html

'<html><head>\n\t<meta name="google-site-verification" content="uvLrphPZotcDpspdcgkCoo8NypsIGl309A67wMoK0qY">\n</head>\n<frameset rows="100%,*">\n\t<frame src="/p00/A0001M002.nice">\n</frameset>\n</html>'

In [10]:
soup = BeautifulSoup(html, 'html.parser'); soup
#soup_lxml = BeautifulSoup(html, 'lxml'); soup_lxml

<html><head>
<meta content="uvLrphPZotcDpspdcgkCoo8NypsIGl309A67wMoK0qY" name="google-site-verification"/>
</head>
<frameset rows="100%,*">
<frame src="/p00/A0001M002.nice"/>
</frameset>
</html>

In [11]:
# BeautifulSoup(html, 'html.parser').find('frame').get('src')
soup.find('frame')
soup.find('frame').get('src')

'/p00/A0001M002.nice'

In [12]:
url_frame = url + soup.find('frame').get('src')

driver.get(url_frame)
html = driver.page_source

In [13]:
soup = BeautifulSoup(html, 'html.parser'); # soup
# soup_lxml = BeautifulSoup(html, 'lxml'); soup_lxml

## (2) Tag 찾아보기 : 공지사항 내용 

In [14]:
soup.find('div', attrs = {'class': 'board'}).find('ul')

<ul>
<li><a href="javascript:notice('90');">
									[중요공지] RM1 서비스가 NICE Bi...
								</a><span class="date">
									2022.11.10
								</span></li>
<li><a href="javascript:notice('89');">
									[당첨자 발표] ValueSearch(Ex...
								</a><span class="date">
									2022.10.19
								</span></li>
</ul>

In [15]:
soup.find('div', attrs = {'class': 'board'}).find('ul').getText()

'\n\n\t\t\t\t\t\t\t\t\t[중요공지] RM1 서비스가 NICE Bi...\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t2022.11.10\n\t\t\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\t\t[당첨자 발표] ValueSearch(Ex...\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t2022.10.19\n\t\t\t\t\t\t\t\t\n'

#### 불필요한 문자열 제거 :

In [16]:
soup.find('div', attrs = {'class': 'board'}).find('ul').getText().strip('\n\t')

'[중요공지] RM1 서비스가 NICE Bi...\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t2022.11.10\n\t\t\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\t\t[당첨자 발표] ValueSearch(Ex...\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t2022.10.19'

In [17]:
import re

re.sub(r"[\n\t\s]*", "", soup.find('div', attrs = {'class': 'board'}).find('ul').getText())

'[중요공지]RM1서비스가NICEBi...2022.11.10[당첨자발표]ValueSearch(Ex...2022.10.19'

## (3) 팝업창 Web crawling

####  [기업정보/신용정보] 클릭Click 을 통해 새로운 창이 떴을 경우,

**1) New window(새창) 으로 Focus 이동:**

In [18]:
# posting = driver.find_element_by_xpath('//*[@id="main_cont"]/li[2]/dl/dd[1]')
# posting.click()

In [19]:
# driver.switch_to_window(driver.window_handles[1])  # [1] : Click 을 통해 띄워지는 New window (새창) 의미
### new version
driver.switch_to.window(driver.window_handles[1]) # 새로운 창으로 focus를 이동

**2) New window 의 html 소스 가져오기**

In [20]:
driver.page_source

'<html xmlns="http://www.w3.org/1999/xhtml" lang="ko" xml:lang="ko"><head>\n<meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge"> \n<meta http-equiv="expires" content="-1">\n<title> RM</title>\n<link rel="stylesheet" type="text/css" media="all" href="/common/css/base.css">\n<link rel="stylesheet" type="text/css" media="all" href="/common/css/layout.css?date=20220221">\n<link rel="stylesheet" type="text/css" media="all" href="/common/css/module.css?date=20220221">\n<link rel="stylesheet" type="text/css" media="all" href="/common/css/section_a.css">\n<link rel="stylesheet" type="text/css" media="all" href="/common/css/section_b.css">\n<link rel="stylesheet" type="text/css" media="all" href="/common/css/comm.css">\n<link rel="stylesheet" type="text/css" media="all" href="/common/css/pop.css">\n<link rel="stylesheet" type="text/css" media="all" href="/common/css/calendar.css">\n<link rel="stylesheet" type="text/css" media

**3) New window 닫기 close**

In [21]:
driver.close()

**4)원래 Main window 로 Focus 이동**

In [22]:
#driver.switch_to_window(driver.window_handles[0])  # [0] : 원래 창(부모), Main window 의미
### new version
driver.switch_to.window(driver.window_handles[0]) 

In [23]:
# driver.page_source

참고 : [Python Selenium 사용법 [파이썬 셀레늄 사용법, 크롤링]](https://greeksharifa.github.io/references/2020/10/30/python-selenium-usage/)